#### 질의 변형(Query reformation) 

In [1]:
# 질의 변형 : 사용자의 원래 질문을 보다 효과적인 검색 쿼리로 변환하는 과정 (p. 216~)
# 사용자의 질문이 모호하거나 검색에 최적화되지 않은 형태의 경우, 연관 문서를 제대로 검색할 수 없어 RAG 시스템의 정확도가 떨어짐
# 원래의 질문을 분석하고 재구성 :
# - 질문을 더 구체적이고 검색에 최적화된 형태로 재작성
# - 복잡한 질문을 여러 개의 하위 질문으로 분해하는 등의 기법 사용
# - 잘문의 의도를 파악하여 더 일반화된 쿼리로 확장하거나,
# - 가상의 문서를 생성하여 갬색 성능을 개선하는 방법도 포함

####- 1) MultiQuery Generation (250622) : 다중 질의 생성

In [ ]:
# 질문 -> 생성 (Q1, Q2, Q3...) -> DB -> 문서 검색 (doc1, doc2, doc3...) -> LLM -> 답변

# 질문 : 주식 투자를 처음 시작하려면 어떻게 해야 하나요?

# Q1 : 초보 투자자를 위한 주식 투자 기초 지식은 무엇인가요?
# Q2 : 주식 시장 분석을 위한 기본적인 재무제표 읽는 법은?
# Q3 : 주식 투자 시작 전 필요한 자금 관리 전략은?

In [1]:
# 다중 질의 생성 과정을 모니터링하기 위한 로깅 설정
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [2]:
# 디렉토리 내 모든 파일을 리스트로 변환하는 함수 정의

import os

def list_files(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

# 지정된 디렉토리 내 모든 파일명을 리스트로 호출
file_names = list_files('./data')
print(file_names)


['./data/.DS_Store', './data/constitution_of_Korea.pdf']


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',    # 다국어 모델
    # model_name='jhgan/ko-sroberta-multitask',  # 한국어 모델 - 에러 발생 (250603)
    # model_name = 'BAAI/bge-m3',                # 에러 발생 (250603)
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

embeddings_model

/var/folders/kv/jhs4vb392hb9h62z_b83xjwm0000gn/T/ipykernel_7946/82237342.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

In [4]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

loader = PyMuPDFLoader(file_names[1])       # 폴더 내 파일 1개만 존재 : 여러 개일 경우, 최초 1개 DB 생성 후, Add 방식으로 진행 (250605)
documents = loader.load()

# 맥의 경우 .ds 파일이 생성되므로 인덱스를 [1]부터 이용한다. (250620)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=16) 
docs = text_splitter.split_documents(documents)

# 임베딩 DB 생성 : 파일로 저장하지 않으므로, 새로 실행할 경우 초기화됨 (250605)
# Chroma vevtorstore 생성 (p. 220 까지)
db_constitution = Chroma.from_documents(
    documents=docs, embedding=embeddings_model, collection_name="db_constitution"
)

In [5]:
from langchain.retrievers import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# LLM 모델 설정 (Local LM Studio 사용, 250622)
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    # model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    model = "lmstudio-community/gemma-2-2b-it-GGUF",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()], # 스트림 출력 콜백
)

# MultiQueryRetriever 실행
retriever = MultiQueryRetriever.from_llm(
    retriever = db_constitution.as_retriever(), # 기본 검색기 (벡터 데이터베이스)
    llm = llm,    # 위에서 정의한 llm 사용
)


In [9]:
# Query 작성
question = "국민의 의무에 관하여 남성과 여성의 차이가 있나요? 한국어로 답변해줘."

# 결과 검색
unique_docs = retriever.invoke(question)
print(f"\n결과: {len(unique_docs)}개의 문서가 검색되었습니다.")

Here are three alternative versions of the user's question, designed to address potential limitations of distance-based similarity search in a vector database:

1. **"남성과 여성의 의무에 대한 차이는 어떻게 나타나요? 국민의 의무와 관련된 사회적 관점에서 분석해 주세요."** (This version focuses on the social implications and analysis of gender differences in obligations, potentially broadening the search beyond simple binary comparisons.)

2. **"국민의 의무에 대한 차이가 남성과 여성 사이에 존재하는지, 그리고 그 차이는 어떤 요인에 의해 발생하는지에 대해 설명해 주세요."** (This version emphasizes the factors contributing to gender differences in obligations, potentially leading to a more nuanced search.)

3. **"국민의 의무에 대한 차이가 남성과 여성 사이에 존재하는데, 이러한 차이는 사회적 기준이나 문화적 요인에 영향을 받는지 분석해 주세요."** (This version explores the influence of social and cultural factors on gender differences in obligations, potentially uncovering more complex relationships.) 


These alternative questions aim to:

* **Broaden the scope:**  They move beyond a simple binary comparison of "male vs. female" and delve i

INFO:langchain.retrievers.multi_query:Generated queries: ["Here are three alternative versions of the user's question, designed to address potential limitations of distance-based similarity search in a vector database:", '1. **"남성과 여성의 의무에 대한 차이는 어떻게 나타나요? 국민의 의무와 관련된 사회적 관점에서 분석해 주세요."** (This version focuses on the social implications and analysis of gender differences in obligations, potentially broadening the search beyond simple binary comparisons.)', '2. **"국민의 의무에 대한 차이가 남성과 여성 사이에 존재하는지, 그리고 그 차이는 어떤 요인에 의해 발생하는지에 대해 설명해 주세요."** (This version emphasizes the factors contributing to gender differences in obligations, potentially leading to a more nuanced search.)', '3. **"국민의 의무에 대한 차이가 남성과 여성 사이에 존재하는데, 이러한 차이는 사회적 기준이나 문화적 요인에 영향을 받는지 분석해 주세요."** (This version explores the influence of social and cultural factors on gender differences in obligations, potentially uncovering more complex relationships.) ', 'These alternative questions aim to:', '* **Broaden the scope:**  They mov


결과: 15개의 문서가 검색되었습니다.


In [ ]:
# LLM model : gemma-2-2b-it - 1.71 GB

# 국민의 의무에 관하여 남성과 여성의 차이가 있나요? 라는 기존 질문에 대하여 
# 추가로 3가지 다른 관점의 질문들로 변환되어 검색에 활용된다.

# 번역:
'''
"How does gender influence compliance with mandatory civic duties in South Korea?"
→ "한국에서 성별은 의무적인 시민의무 이행에 어떤 영향을 미치는가?"
이 문장은 특정 사회적 맥락에 초점을 맞추며 성별 간의 미묘한 차이를 포착하려는 목적을 갖고 있습니다.

"What are the key factors contributing to variations in civic duty participation between men and women in Korean society?"
→ "한국 사회에서 남성과 여성 간 시민의무 참여 차이에 영향을 미치는 주요 요인은 무엇인가?"
이 질문은 단순한 성별 비교를 넘어, 그러한 차이에 영향을 주는 근본적인 요인에 중점을 둡니다.

"Can you identify any significant gender disparities in the fulfillment of mandatory civic obligations within South Korea's population?"
→ "한국 국민의 의무적인 시민의무 이행에서 뚜렷한 성별 격차가 존재하는가?"
이 문장은 보다 직접적인 표현을 사용하여 원하는 정보를 강조하며, "차이(differences)"와 같은 모호한 표현을 피하고 있습니다.
'''

# "모호한 표현을 피하고 있다" 라는 표현에 주목하자. 
# ChatGPT를 통해 번역한 결과이며, 추가적인 주석을 포함하여 상세하게 설명해주고 있다.

# 정확하게 질문하는 방법에 대한 제시
# 내가 하는 질문에 문제가 없는지 먼저 검토할 수 있는 가능성 : 최초 질문 -> 질문의 확장

In [10]:
from langchain.chains import RetrievalQA

# RetrivalQA 체인 설정
 
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,      # MultiQueryRetrivever 전달
    return_source_documents = True
)

# 질문에 대한 답변 생성
result = qa_chain.invoke({"query" : question})

# 결과 출력
print("답변 : ", result["result"])
print("\n사용된 문서:")
for doc in result["source_documents"]:
    print(doc.page_content)


Here are three alternative versions of the user's question, designed to improve retrieval in a vector database:

1. **"남녀 간 의무에 대한 차이점은 무엇인가요?"** (What are the differences in mandatory duties between men and women?) 
    * This version focuses on the "difference" aspect of the query, potentially capturing more nuanced semantic relationships than just a simple comparison.

2. **"국민의 의무에 대한 성별 차이를 분석한 연구는 어떤 내용을 담고 있나요?"** (What research analyzes gender differences in mandatory duties?)
    * This version emphasizes the "research" aspect, potentially retrieving documents that go beyond just a simple statement and include analysis or studies.

3. **"국민 의무에 대한 사회적 관점에서 남녀의 차이가 존재하는지 분석해 보세요."** (Analyze the social perspective on differences in mandatory duties between men and women.)
    * This version focuses on the "social perspective" aspect, potentially retrieving documents that discuss broader societal implications of gender-based differences.


These alternative questions are designe

INFO:langchain.retrievers.multi_query:Generated queries: ["Here are three alternative versions of the user's question, designed to improve retrieval in a vector database:", '1. **"남녀 간 의무에 대한 차이점은 무엇인가요?"** (What are the differences in mandatory duties between men and women?) ', '    * This version focuses on the "difference" aspect of the query, potentially capturing more nuanced semantic relationships than just a simple comparison.', '2. **"국민의 의무에 대한 성별 차이를 분석한 연구는 어떤 내용을 담고 있나요?"** (What research analyzes gender differences in mandatory duties?)', '    * This version emphasizes the "research" aspect, potentially retrieving documents that go beyond just a simple statement and include analysis or studies.', '3. **"국민 의무에 대한 사회적 관점에서 남녀의 차이가 존재하는지 분석해 보세요."** (Analyze the social perspective on differences in mandatory duties between men and women.)', '    * This version focuses on the "social perspective" aspect, potentially retrieving documents that discuss broader societal implicati

한국에서는 남녀 모두 국민의 의무를 수행해야 합니다.  하지만, 특정한 경우에는 남녀 간 차이점이 존재할 수 있습니다. 예를 들어, 여성은 임신 중이나 출산 후에 휴식을 취하는 시간이 필요하거나, 부모로서 아이 돌봄에 집중해야 할 때가 있기 때문에 일부 국민의 의무와 관련하여 특별한 경우 발생할 수 있습니다. 

그러나, 모든 국민에게 동일하게 적용되는 법률과 정책은 존재합니다.  국민의 의무는 남녀 모두에게 공평하게 부여되며, 이에 대한 차이점은 사회적 요인이나 개인적인 상황에 따라 다를 수 있습니다. 
답변 :  한국에서는 남녀 모두 국민의 의무를 수행해야 합니다.  하지만, 특정한 경우에는 남녀 간 차이점이 존재할 수 있습니다. 예를 들어, 여성은 임신 중이나 출산 후에 휴식을 취하는 시간이 필요하거나, 부모로서 아이 돌봄에 집중해야 할 때가 있기 때문에 일부 국민의 의무와 관련하여 특별한 경우 발생할 수 있습니다. 

그러나, 모든 국민에게 동일하게 적용되는 법률과 정책은 존재합니다.  국민의 의무는 남녀 모두에게 공평하게 부여되며, 이에 대한 차이점은 사회적 요인이나 개인적인 상황에 따라 다를 수 있습니다. 


사용된 문서:
14. 정당해산의 제소
15. 정부에 제출 또는 회부된 정부의 정책에 관계되는 청원의 심사
16. 검찰총장ㆍ합동참모의장ㆍ각군참모총장ㆍ국립대학교총장ㆍ대사 기타 법률이 정한 공무
원과 국영기업체관리자의 임명
17. 기타 대통령ㆍ국무총리 또는 국무위원이 제출한 사항
 
제90조 ①국정의 중요한 사항에 관한 대통령의 자문에 응하기 위하여 국가원로로 구성되는 국가
원로자문회의를 둘 수 있다.
한 동의권을 가진다.
 
제61조 ①국회는 국정을 감사하거나 특정한 국정사안에 대하여 조사할 수 있으며, 이에 필요한
서류의 제출 또는 증인의 출석과 증언이나 의견의 진술을 요구할 수 있다.
②국정감사 및 조사에 관한 절차 기타 필요한 사항은 법률로 정한다.
 
제62조 ①국무총리ㆍ국무위원 또는 정부위원은 국회나 그 위원회에 출석하여 국정처리상황을
